In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error

optuna.logging.set_verbosity(optuna.logging.WARNING)

# =========================
# Config (RANDOM)
# =========================
DATA_PATH = "../final_data/data_260129_random.csv"
DATE_COL = "day"

Y_TRIP = "trip_cnt_per_user"
Y_NONR = "nonrepeat_cnt_per_user"

TEST_START = pd.Timestamp("2026-01-05")
TEST_END   = pd.Timestamp("2026-01-11")  # inclusive

# RANDOM 的 group keys：treatment x ops_type_merged x city_group = 8*8*3=192 groups
GROUP_KEYS = ["treatment", "ops_type_merged", "city_group"]
CAT_COLS   = ["treatment", "ops_type_merged", "city_group"]

# ✅ 白名單特徵：RANDOM 要把 treatment 放進模型
LAG_DAYS = list(range(1, 15))  # list(range(10, 22)) = 10..21 inclusive
#LAG_DAYS = [14, 21]
LAG_VARS = ["trip_cnt_per_user", "nonrepeat_cnt_per_user", "match_rate", "coupon_BD_per_user", "coupon_CDP_per_user", "coupon_folk_per_user",
    "coupon_growth_other_per_user", "coupon_MGM_per_user", "coupon_MKT_per_user",
    "coupon_register_per_user", "coupon_daily_per_user"]

BASE_FEATURES = (
    ["treatment", "ops_type_merged", "city_group",
     "is_weekend_holiday", "is_rainy", "mgm_day"]
    + [f"{v}_lag{d}" for v in LAG_VARS for d in LAG_DAYS]
)

# 你目前手動微調後的參數（baseline / center）
PARAMS_TRIP_BASE = dict(
    objective="regression",
    metric="rmse",
    learning_rate=0.04,
    num_leaves=31,
    min_data_in_leaf=200,
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=1,
    lambda_l2=1.0,
    seed=42,
    verbosity=-1,
)

PARAMS_NONR_BASE = dict(
    objective="regression",
    metric="rmse",
    learning_rate=0.05,
    num_leaves=31,
    min_data_in_leaf=50,
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=1,
    lambda_l2=1.0,
    seed=42,
    verbosity=-1,
)

NUM_BOOST_ROUND_TRIP = 10000
NUM_BOOST_ROUND_NONR = 10000
EARLY_STOPPING_TRIP = 200
EARLY_STOPPING_NONR = 200
VALID_DAYS = 28

N_TRIALS = 120  # per model

# =========================
# Helpers
# =========================
def rmse(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    return float(np.sqrt(mean_squared_error(y_true[mask], y_pred[mask])))

def weekly_sum_rmse_groups(test_df, pred_df, target_col, pred_col, expected_groups: int):
    """
    Weekly-sum RMSE across expected_groups groups.
    test_df: must contain GROUP_KEYS + DATE_COL + target_col
    pred_df: must contain GROUP_KEYS + DATE_COL + pred_col
    """
    eval_df = (
        test_df[GROUP_KEYS + [DATE_COL, target_col]]
        .merge(
            pred_df[GROUP_KEYS + [DATE_COL, pred_col]],
            on=GROUP_KEYS + [DATE_COL],
            how="left",
            validate="one_to_one",
        )
    )
    if eval_df[pred_col].isna().any():
        raise ValueError(f"Missing predictions after merge for {pred_col}")

    n_groups = eval_df.groupby(GROUP_KEYS, dropna=False).ngroups
    if n_groups != expected_groups:
        raise ValueError(f"Expected {expected_groups} groups, got {n_groups}")

    g = (
        eval_df
        .groupby(GROUP_KEYS, dropna=False)
        .agg(actual_sum=(target_col, "sum"), pred_sum=(pred_col, "sum"))
        .reset_index()
    )
    return float(np.sqrt(np.mean((g["actual_sum"] - g["pred_sum"]) ** 2)))

def align_categories_for_train_test(train_df, test_df, cat_cols):
    for c in cat_cols:
        if c not in train_df.columns or c not in test_df.columns:
            continue
        tr = train_df[c].astype("object")
        te = test_df[c].astype("object")
        cats = pd.Index(tr.dropna().unique()).union(pd.Index(te.dropna().unique()))
        train_df[c] = pd.Categorical(tr, categories=cats)
        test_df[c]  = pd.Categorical(te, categories=cats)
    return train_df, test_df

def get_feature_cols(df):
    feats = [c for c in BASE_FEATURES if c in df.columns]
    # 防呆：同日 target 永不進特徵
    feats = [c for c in feats if c not in [Y_TRIP, Y_NONR]]
    cat = [c for c in CAT_COLS if c in feats]
    return feats, cat

def time_based_train_valid_split(train_df, valid_days=28):
    cutoff = train_df[DATE_COL].max() - pd.Timedelta(days=valid_days)
    tr_sub = train_df[train_df[DATE_COL] < cutoff].copy()
    va_sub = train_df[train_df[DATE_COL] >= cutoff].copy()
    return tr_sub, va_sub

def build_params_from_trial_narrow(trial: optuna.Trial, base_params: dict, target: str):
    """
    Narrow search space around your hand-tuned base.
    (Same policy as control version)
    """
    params = dict(base_params)

    base_lr = float(base_params.get("learning_rate", 0.05))
    lr_low  = max(0.01, base_lr * 0.6)
    lr_high = min(0.12, base_lr * 1.6)
    params["learning_rate"] = trial.suggest_float("learning_rate", lr_low, lr_high, log=True)

    params["num_leaves"] = trial.suggest_int("num_leaves", 15, 63, step=2)

    if target == Y_TRIP:
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 80, 350, step=10)
    else:
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 20, 120, step=5)

    params["max_depth"] = trial.suggest_categorical("max_depth", [-1, 4, 5, 6, 7, 8, 9, 10])

    params["feature_fraction"] = trial.suggest_float("feature_fraction", 0.75, 1.0, step=0.05)
    params["bagging_fraction"] = trial.suggest_float("bagging_fraction", 0.75, 1.0, step=0.05)
    params["bagging_freq"] = trial.suggest_categorical("bagging_freq", [0, 1, 2, 5])

    params["lambda_l2"] = trial.suggest_float("lambda_l2", 0.2, 6.0, log=True)
    params["lambda_l1"] = trial.suggest_float("lambda_l1", 0.0, 2.0)

    params["min_gain_to_split"] = trial.suggest_float("min_gain_to_split", 0.0, 0.05)

    params["extra_trees"] = trial.suggest_categorical("extra_trees", [False, True])

    params["objective"] = "regression"
    params["metric"] = "rmse"
    params["seed"] = 42
    params["verbosity"] = -1

    return params

# =========================
# Load data + split
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.replace({"": np.nan})

# Train is strictly before test week start
train_all = df[df[DATE_COL] < TEST_START].copy()

# The week you want to evaluate/tune on
test_week = df[(df[DATE_COL] >= TEST_START) & (df[DATE_COL] <= TEST_END)].copy()

feature_cols, cat_cols = get_feature_cols(df)

# Align categories for train/test week
train_all, test_week = align_categories_for_train_test(train_all, test_week, cat_cols)

# internal valid split for early stopping (only within train_all)
tr_sub, va_sub = time_based_train_valid_split(train_all, valid_days=VALID_DAYS)
tr_sub, va_sub = align_categories_for_train_test(tr_sub, va_sub, cat_cols)

# expected number of groups in random: 8*8*3 = 192 (you can hardcode 192)
EXPECTED_GROUPS = 192

# =========================
# Objective factory: minimize weekly-sum RMSE on 1/5-1/11 across 192 groups
# =========================
def make_objective_weeksum(target: str, base_params: dict, num_boost_round: int, early_stop: int):
    X_tr = tr_sub[feature_cols]
    y_tr = tr_sub[target].astype(float)
    X_va = va_sub[feature_cols]
    y_va = va_sub[target].astype(float)

    dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_cols, free_raw_data=False)
    dvalid = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_cols, free_raw_data=False)

    X_test = test_week[feature_cols]

    def objective(trial: optuna.Trial) -> float:
        params = build_params_from_trial_narrow(trial, base_params, target)

        model = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=num_boost_round,
            valid_sets=[dvalid],
            callbacks=[lgb.early_stopping(stopping_rounds=early_stop, verbose=False)],
        )

        # predict 1/5-1/11 (expected rows: 7*192=1344)
        pred = model.predict(X_test, num_iteration=model.best_iteration)

        pred_df = test_week[GROUP_KEYS + [DATE_COL]].copy()
        pred_col = f"pred_{target}"
        pred_df[pred_col] = pred

        score = weekly_sum_rmse_groups(
            test_df=test_week,
            pred_df=pred_df,
            target_col=target,
            pred_col=pred_col,
            expected_groups=EXPECTED_GROUPS,
        )

        trial.set_user_attr("best_iteration", int(model.best_iteration))
        return score

    return objective

# =========================
# Run studies (120 trials each)
# =========================
sampler = optuna.samplers.TPESampler(seed=42)

# --- TRIP
study_trip = optuna.create_study(direction="minimize", sampler=sampler)
study_trip.optimize(
    make_objective_weeksum(Y_TRIP, PARAMS_TRIP_BASE, NUM_BOOST_ROUND_TRIP, EARLY_STOPPING_TRIP),
    n_trials=N_TRIALS,
    show_progress_bar=True
)
print("\n=== BEST (TRIP | weekly-sum RMSE across 192 groups) ===")
print("best value:", study_trip.best_value)
print("best params:", study_trip.best_params)
print("best_iteration:", study_trip.best_trial.user_attrs.get("best_iteration"))

# --- NONREPEAT
study_nonr = optuna.create_study(direction="minimize", sampler=sampler)
study_nonr.optimize(
    make_objective_weeksum(Y_NONR, PARAMS_NONR_BASE, NUM_BOOST_ROUND_NONR, EARLY_STOPPING_NONR),
    n_trials=N_TRIALS,
    show_progress_bar=True
)
print("\n=== BEST (NONREPEAT | weekly-sum RMSE across 192 groups) ===")
print("best value:", study_nonr.best_value)
print("best params:", study_nonr.best_params)
print("best_iteration:", study_nonr.best_trial.user_attrs.get("best_iteration"))

# =========================
# Merge best params back to full params dict
# =========================
def merge_best(base_params, best_params):
    merged = dict(base_params)
    merged.update(best_params)
    merged["objective"] = "regression"
    merged["metric"] = "rmse"
    merged["seed"] = 42
    merged["verbosity"] = -1
    return merged

BEST_PARAMS_TRIP = merge_best(PARAMS_TRIP_BASE, study_trip.best_params)
BEST_PARAMS_NONR = merge_best(PARAMS_NONR_BASE, study_nonr.best_params)

print("\nBEST_PARAMS_TRIP =", BEST_PARAMS_TRIP)
print("BEST_PARAMS_NONR =", BEST_PARAMS_NONR)


  0%|          | 0/120 [00:00<?, ?it/s]

/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py:102: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  n_groups = eval_df.groupby(GROUP_KEYS, dropna=False).ngroups
/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py:107: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  eval_df
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
[LightGBM] [Fatal] 


=== BEST (TRIP | weekly-sum RMSE across 192 groups) ===
best value: 0.03111380838963007
best params: {'learning_rate': 0.028996871918837788, 'num_leaves': 61, 'min_data_in_leaf': 250, 'max_depth': 8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'lambda_l2': 4.990682103388769, 'lambda_l1': 0.05468471712199191, 'min_gain_to_split': 2.5970471043495818e-05, 'extra_trees': False}
best_iteration: 199


/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py:102: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  n_groups = eval_df.groupby(GROUP_KEYS, dropna=False).ngroups
/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py:107: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  eval_df


  0%|          | 0/120 [00:00<?, ?it/s]

/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py:102: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  n_groups = eval_df.groupby(GROUP_KEYS, dropna=False).ngroups
/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py:107: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  eval_df
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
[LightGBM] [Fatal] 

[W 2026-01-30 16:24:05,077] Trial 69 failed with parameters: {'learning_rate': 0.06078469131374224, 'num_leaves': 45, 'min_data_in_leaf': 100, 'max_depth': 5, 'feature_fraction': 0.85, 'bagging_fraction': 1.0, 'bagging_freq': 2, 'lambda_l2': 2.3913293489999217, 'lambda_l1': 0.864644462392876, 'min_gain_to_split': 0.0024948451285523258, 'extra_trees': True} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/shotime/Desktop/2026winter_project/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 206, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/1739132621.py", line 219, in objective
    model = lgb.train(
  File "/Users/shotime/Desktop/2026winter_project/.venv/lib/python3.10/site-packages/lightgbm/engine.py", line 322, in train
    booster.update(fobj=fobj)
  File "/Users/shotime/Desktop/2026winter_project/.venv/lib/python3.10/site-packages/lightgb

KeyboardInterrupt: 

In [8]:
## random comare 1215-1228

import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error

optuna.logging.set_verbosity(optuna.logging.WARNING)

# =========================
# Config
# =========================
DATA_PATH = "../final_data/data_260129_random.csv"
DATE_COL = "day"

Y_TRIP = "trip_cnt_per_user"
Y_NONR = "nonrepeat_cnt_per_user"

TEST_START = pd.Timestamp("2025-12-15")
TEST_END   = pd.Timestamp("2025-12-28")  # inclusive

OPS_TREAT_RULES = {
    "14天在其他尖峰預估車資": [15, 20],
    "14天在晚尖峰預估車資":   [15, 20],
    "90天在尖峰預估車資":     [20, 30],
    "喚回-其他":              [20, 30],
    "喚回-高優惠敏感":        [20, 30],
    "既有regular鞏固":        [15, 20],
    "養成Regular-其他":       [20, 30],
    "養成Regular-高優惠敏感": [20, 30],
}

EVAL_GROUP_KEYS = ["city_group", "ops_type_merged", "treatment"]
CAT_COLS = ["treatment", "ops_type_merged", "city_group"]

LAG_DAYS = list(range(7, 22))
LAG_VARS = ["trip_cnt_per_user", "nonrepeat_cnt_per_user", "match_rate", "coupon_BD_per_user", "coupon_CDP_per_user", "coupon_folk_per_user",
    "coupon_growth_other_per_user", "coupon_MGM_per_user", "coupon_MKT_per_user",
    "coupon_register_per_user", "coupon_daily_per_user"]

BASE_FEATURES = (
    ["treatment", "ops_type_merged", "city_group",
     "is_weekend_holiday", "is_rainy", "mgm_day", "face_value", "face_value_num"]
    + [f"{v}_lag{d}" for v in LAG_VARS for d in LAG_DAYS]
)

# baseline centers (optuna 會在附近搜尋)
PARAMS_TRIP_BASE = dict(
    objective="regression",
    metric="rmse",
    learning_rate=0.05,
    num_leaves=63,
    min_data_in_leaf=200,
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=1,
    lambda_l2=1.0,
    seed=42,
    verbosity=-1,
)

PARAMS_NONR_BASE = dict(
    objective="regression",
    metric="rmse",
    learning_rate=0.05,
    num_leaves=31,
    min_data_in_leaf=50,
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=1,
    lambda_l2=1.0,
    seed=42,
    verbosity=-1,
)

NUM_BOOST_ROUND_TRIP = 5000
NUM_BOOST_ROUND_NONR = 5000
EARLY_STOPPING_TRIP = 200
EARLY_STOPPING_NONR = 200
VALID_DAYS = 28

N_TRIALS = 120
FILL_MISSING_WITH_ZERO = True


# =========================
# Helpers
# =========================
def align_categories(train_df: pd.DataFrame, test_df: pd.DataFrame, cat_cols):
    for c in cat_cols:
        if c not in train_df.columns or c not in test_df.columns:
            continue
        tr = train_df[c].astype("object")
        te = test_df[c].astype("object")
        cats = pd.Index(tr.dropna().unique()).union(pd.Index(te.dropna().unique()))
        train_df[c] = pd.Categorical(tr, categories=cats)
        test_df[c]  = pd.Categorical(te, categories=cats)
    return train_df, test_df

def get_feature_cols(df: pd.DataFrame):
    feats = [c for c in BASE_FEATURES if c in df.columns]
    feats = [c for c in feats if c not in [Y_TRIP, Y_NONR]]
    cat = [c for c in CAT_COLS if c in feats]
    return feats, cat

def time_based_train_valid_split(train_df: pd.DataFrame, valid_days=28):
    cutoff = train_df[DATE_COL].max() - pd.Timedelta(days=valid_days)
    tr_sub = train_df[train_df[DATE_COL] < cutoff].copy()
    va_sub = train_df[train_df[DATE_COL] >= cutoff].copy()
    return tr_sub, va_sub

def build_allowed_pairs(rules: dict) -> pd.DataFrame:
    rows = []
    for ops, fvs in rules.items():
        for fv in fvs:
            rows.append({"ops_type_merged": ops, "treatment": f"{int(fv)}元1張"})
    return pd.DataFrame(rows).drop_duplicates()

def build_expected_eval_groups(df_test_all: pd.DataFrame, rules: dict, eval_group_keys: list) -> pd.DataFrame:
    allowed_pairs = build_allowed_pairs(rules)

    cities = (
        df_test_all[["city_group"]]
        .drop_duplicates()
        .assign(_k=1)
    )

    base_groups = (
        cities.merge(allowed_pairs.assign(_k=1), on="_k", how="inner")
        .drop(columns=["_k"])
    )

    windows = pd.DataFrame({"week_window": ["w1_1215_1221", "w2_1222_1228"]}).assign(_k=1)

    expected = (
        base_groups.assign(_k=1)
        .merge(windows, on="_k", how="inner")
        .drop(columns=["_k"])
    )
    return expected[eval_group_keys + ["week_window"]].drop_duplicates()

def two_week_sum_rmse_on_subset_city(
    df_test_all: pd.DataFrame,
    target_col: str,
    pred_col: str,
    rules: dict,
    eval_group_keys: list,
    fill_missing_with_zero: bool = True,
) -> float:
    allowed_pairs = build_allowed_pairs(rules)

    sub = df_test_all.merge(allowed_pairs, on=["ops_type_merged", "treatment"], how="inner")
    sub = sub[(sub[DATE_COL] >= TEST_START) & (sub[DATE_COL] <= TEST_END)].copy()

    d = sub[DATE_COL]
    cond1 = (d >= pd.Timestamp("2025-12-15")) & (d <= pd.Timestamp("2025-12-21"))
    cond2 = (d >= pd.Timestamp("2025-12-22")) & (d <= pd.Timestamp("2025-12-28"))

    sub["week_window"] = pd.NA
    sub.loc[cond1, "week_window"] = "w1_1215_1221"
    sub.loc[cond2, "week_window"] = "w2_1222_1228"
    sub = sub[sub["week_window"].notna()].copy()

    if sub[target_col].isna().any():
        miss = sub[sub[target_col].isna()][eval_group_keys + [DATE_COL]].head(10)
        raise ValueError(f"Missing ground truth in subset for {target_col}. Sample:\n{miss}")
    if sub[pred_col].isna().any():
        miss = sub[sub[pred_col].isna()][eval_group_keys + [DATE_COL]].head(10)
        raise ValueError(f"Missing prediction in subset for {pred_col}. Sample:\n{miss}")

    g = (
        sub
        .groupby(eval_group_keys + ["week_window"], dropna=False)
        .agg(actual_sum=(target_col, "sum"), pred_sum=(pred_col, "sum"))
        .reset_index()
    )

    expected = build_expected_eval_groups(df_test_all, rules, eval_group_keys)
    idx = pd.MultiIndex.from_frame(expected)
    g2 = g.set_index(eval_group_keys + ["week_window"]).reindex(idx)

    if g2[["actual_sum", "pred_sum"]].isna().any().any():
        if fill_missing_with_zero:
            g2[["actual_sum", "pred_sum"]] = g2[["actual_sum", "pred_sum"]].fillna(0.0)
        else:
            missing_rows = g2[g2["actual_sum"].isna() | g2["pred_sum"].isna()].head(10)
            raise ValueError(f"Missing some (city,ops,treat,week_window) sums. Sample:\n{missing_rows}")

    g2 = g2.reset_index()
    err = (g2["actual_sum"] - g2["pred_sum"]).to_numpy(dtype=float)
    return float(np.sqrt(np.mean(err ** 2)))

def build_params_from_trial(trial: optuna.Trial, base_params: dict, target: str):
    params = dict(base_params)

    base_lr = float(base_params.get("learning_rate", 0.05))
    lr_low  = max(0.01, base_lr * 0.6)
    lr_high = min(0.12, base_lr * 1.6)
    params["learning_rate"] = trial.suggest_float("learning_rate", lr_low, lr_high, log=True)

    params["num_leaves"] = trial.suggest_int("num_leaves", 15, 127, step=2)

    if target == Y_TRIP:
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 50, 400, step=10)
    else:
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 20, 200, step=5)

    params["max_depth"] = trial.suggest_categorical("max_depth", [-1, 4, 5, 6, 7, 8, 9, 10])

    params["feature_fraction"] = trial.suggest_float("feature_fraction", 0.7, 1.0, step=0.05)
    params["bagging_fraction"] = trial.suggest_float("bagging_fraction", 0.7, 1.0, step=0.05)
    params["bagging_freq"] = trial.suggest_categorical("bagging_freq", [0, 1, 2, 5])

    params["lambda_l2"] = trial.suggest_float("lambda_l2", 0.2, 8.0, log=True)
    params["lambda_l1"] = trial.suggest_float("lambda_l1", 0.0, 3.0)

    params["min_gain_to_split"] = trial.suggest_float("min_gain_to_split", 0.0, 0.05)
    params["extra_trees"] = trial.suggest_categorical("extra_trees", [False, True])

    params["objective"] = "regression"
    params["metric"] = "rmse"
    params["seed"] = 42
    params["verbosity"] = -1
    return params


# =========================
# Load & Split
# =========================
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.replace({"": np.nan})

train_all = df[df[DATE_COL] < TEST_START].copy()
test_df   = df[(df[DATE_COL] >= TEST_START) & (df[DATE_COL] <= TEST_END)].copy()

feature_cols, cat_cols = get_feature_cols(df)

# Align categories for train/test
train_all, test_df = align_categories(train_all, test_df, cat_cols)

# internal valid split
tr_sub, va_sub = time_based_train_valid_split(train_all, valid_days=VALID_DAYS)
tr_sub, va_sub = align_categories(tr_sub, va_sub, cat_cols)


# =========================
# Objective factory
# =========================
def make_objective(target: str, base_params: dict, num_boost_round: int, early_stop: int):
    X_tr = tr_sub[feature_cols]
    y_tr = tr_sub[target].astype(float)
    X_va = va_sub[feature_cols]
    y_va = va_sub[target].astype(float)

    dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_cols, free_raw_data=False)
    dvalid = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_cols, free_raw_data=False)

    X_test = test_df[feature_cols]

    def objective(trial: optuna.Trial) -> float:
        params = build_params_from_trial(trial, base_params, target)

        model = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=num_boost_round,
            valid_sets=[dvalid],
            callbacks=[lgb.early_stopping(stopping_rounds=early_stop, verbose=False)],
        )

        pred = model.predict(X_test, num_iteration=model.best_iteration)

        # attach pred to full test_df, then evaluate subset windows RMSE
        test_pred = test_df[[DATE_COL, "city_group", "ops_type_merged", "treatment", target]].copy()
        pred_col = f"pred_{target}"
        test_pred[pred_col] = pred

        score = two_week_sum_rmse_on_subset_city(
            df_test_all=test_pred,
            target_col=target,
            pred_col=pred_col,
            rules=OPS_TREAT_RULES,
            eval_group_keys=EVAL_GROUP_KEYS,
            fill_missing_with_zero=FILL_MISSING_WITH_ZERO,
        )

        trial.set_user_attr("best_iteration", int(model.best_iteration))
        return score

    return objective


# =========================
# Run studies
# =========================
sampler = optuna.samplers.TPESampler(seed=42)

# --- TRIP
study_trip = optuna.create_study(direction="minimize", sampler=sampler)
study_trip.optimize(
    make_objective(Y_TRIP, PARAMS_TRIP_BASE, NUM_BOOST_ROUND_TRIP, EARLY_STOPPING_TRIP),
    n_trials=N_TRIALS,
    show_progress_bar=True
)
print("\n=== BEST (TRIP | subset two-window sum RMSE) ===")
print("best value:", study_trip.best_value)
print("best params:", study_trip.best_params)
print("best_iteration:", study_trip.best_trial.user_attrs.get("best_iteration"))

# --- NONREPEAT
study_nonr = optuna.create_study(direction="minimize", sampler=sampler)
study_nonr.optimize(
    make_objective(Y_NONR, PARAMS_NONR_BASE, NUM_BOOST_ROUND_NONR, EARLY_STOPPING_NONR),
    n_trials=N_TRIALS,
    show_progress_bar=True
)
print("\n=== BEST (NONREPEAT | subset two-window sum RMSE) ===")
print("best value:", study_nonr.best_value)
print("best params:", study_nonr.best_params)
print("best_iteration:", study_nonr.best_trial.user_attrs.get("best_iteration"))

def merge_best(base_params, best_params):
    merged = dict(base_params)
    merged.update(best_params)
    merged["objective"] = "regression"
    merged["metric"] = "rmse"
    merged["seed"] = 42
    merged["verbosity"] = -1
    return merged

BEST_PARAMS_TRIP = merge_best(PARAMS_TRIP_BASE, study_trip.best_params)
BEST_PARAMS_NONR = merge_best(PARAMS_NONR_BASE, study_nonr.best_params)

print("\nBEST_PARAMS_TRIP =", BEST_PARAMS_TRIP)
print("BEST_PARAMS_NONR =", BEST_PARAMS_NONR)


  0%|          | 0/120 [00:00<?, ?it/s]

/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/2330698863.py:173: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sub
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/2330698863.py:173: FutureWarning: The default of observed=False is deprec


=== BEST (TRIP | subset two-window sum RMSE) ===
best value: 0.054287988710218005
best params: {'learning_rate': 0.06506413839228227, 'num_leaves': 39, 'min_data_in_leaf': 220, 'max_depth': 4, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 2, 'lambda_l2': 1.9919880163917987, 'lambda_l1': 0.14849824104725057, 'min_gain_to_split': 0.00471440671367141, 'extra_trees': True}
best_iteration: 56


/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/2330698863.py:173: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sub


  0%|          | 0/120 [00:00<?, ?it/s]

/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/2330698863.py:173: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sub
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/2330698863.py:173: FutureWarning: The default of observed=False is deprec


=== BEST (NONREPEAT | subset two-window sum RMSE) ===
best value: 0.0755842489812409
best params: {'learning_rate': 0.043004048274569495, 'num_leaves': 67, 'min_data_in_leaf': 190, 'max_depth': -1, 'feature_fraction': 0.7, 'bagging_fraction': 0.8999999999999999, 'bagging_freq': 2, 'lambda_l2': 7.433642012824781, 'lambda_l1': 0.618713178411612, 'min_gain_to_split': 0.01978175292998587, 'extra_trees': True}
best_iteration: 81

BEST_PARAMS_TRIP = {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.06506413839228227, 'num_leaves': 39, 'min_data_in_leaf': 220, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 2, 'lambda_l2': 1.9919880163917987, 'seed': 42, 'verbosity': -1, 'max_depth': 4, 'lambda_l1': 0.14849824104725057, 'min_gain_to_split': 0.00471440671367141, 'extra_trees': True}
BEST_PARAMS_NONR = {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.043004048274569495, 'num_leaves': 67, 'min_data_in_leaf': 190, 'feature_fraction': 0.7, 'bagging

/var/folders/5k/wdr6l04n13xbrtz6l93pdqmc0000gn/T/ipykernel_86291/2330698863.py:173: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sub
